In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import os

# Preprocesamiento de la base de datos

In [ ]:
def parse_station_name(station):
  if "-" in station:
    match = re.search(r"(.+)\s[A-Z]n?\s?-", station)
  else:
    match = re.search(r"(.+)\sT-?\d", station)
  return match.group(1).strip() if match else station.strip()



def preprocess_date_order(folder):
  """Función que devuelve, dado un directorio, sus archivos en orden temporal
  con formato %d-%m-%Y."""


  list_dir_complete = os.listdir(folder)
  basename = list_dir_complete[0].split("-")[0]
  list_dir = [("-".join(dir_complete.split("-")[1:])) for dir_complete in list_dir_complete]
  list_dir_sorted =  sorted(list_dir, key = lambda x: datetime.strptime(x[:-4], "%d-%m-%Y"))
  return [basename +"-"+ dir_sorted for dir_sorted in list_dir_sorted]

#creacion diccionario codigo de ruta a nombre ruta

df_rutas_nombres = pd.read_csv("/content/Rutas Ricardo.csv") #cambiar este nombre porque que pena
df_rutas_nombres = df_rutas_nombres.iloc[:300,:]
rutas_a_nombres = dict(zip(df_rutas_nombres.iloc[:,0], df_rutas_nombres.iloc[:,1]))

########################################

def construct_databas(list_folders, base_path):
  """Funcion que genera, dada una lista de archivos csv, una concatenacion de los mismos"""

  base_path = Path(base_path)
  consortion_name = base_path.name
  print(consortion_name)
  df_total = pd.DataFrame()
  for csv_file in list_folders:
    df = pd.read_csv(base_path / csv_file)
    df["nombre_ruta"]= df["ruta"].map(rutas_a_nombres)
    df = df[["fecha",  "NombreParada", "nombre_ruta", "fechaHoraLecturaDato",
                           "estimacionOcupacionAbordo"]]

    df = df.dropna()
    df["NombreParada"] = df["NombreParada"].apply(parse_station_name)
    df["nombre_ruta"] = df["nombre_ruta"].replace("G43 San Mateo C.C Unisur", "G43 San Mateo C.C. Unisur")
    df["fechaHoraLecturaDato"] = pd.to_datetime(df["fechaHoraLecturaDato"], format="ISO8601").dt.strftime("%H:%M:%S")
    print(df["NombreParada"].unique())
    df_total = pd.concat([df_total, df], ignore_index=True)
  df_total= df_total.sort_values(by=["fecha", "NombreParada", "nombre_ruta"], ascending=[True, True, True])
  df_total.to_csv(f"df_total_{consortion_name}.csv", index=False)




operator_files = preprocess_date_order("/content/drive/Shareddrives/grafos/Datos Red Neuronal de Grafos/SOMOS U")

construct_databas(operator_files, "/content/drive/Shareddrives/grafos/Datos Red Neuronal de Grafos/SOMOS U")

SOMOS U
['Banderas' 'Marsella' 'Av. 39' 'Portal Américas' 'Calle 72' 'Flores'
 'Patio Bonito' 'Calle 57' 'Calle 19' 'Pepe Sierra' 'Virrey' 'Av. 68'
 'Cra. 90' 'Cra. 53' 'Granja - Cra. 77' 'Cra. 47' 'Calle 146' 'Av. Cali'
 'Portal 80' 'Alcalá' 'Calle 106' 'Toberín' 'Calle 85' 'Portal Norte'
 'Calle 100' 'Calle 63' 'Héroes' 'Calle 76' 'Santa Lucía' 'Hortúa' 'Prado'
 'Consuelo' 'Socorro' 'Fucha' 'Marly' 'Calle 142' 'Av. Jiménez'
 'Portal Usme' 'Salitre - El Greco' 'Calle 127' 'Av. Rojas'
 'El Tiempo - Maloka' 'Gobernación' 'Ciudad Universitaria'
 'Concejo de Bogotá' 'Calle 45' 'CAN' 'Calle 26' 'Portal Eldorado'
 'Universidades' 'Polo' 'Minuto de Dios' 'Escuela Militar'
 'Tercer Milenio' 'Calle 187' 'Calle 161' 'Mazurén' 'Guatoque - Veraguas'
 'Terminal' 'Tygua - San José' 'Calle 22' 'Calle 34' 'Movistar Arena'
 'Paloquemao' 'Nariño' 'Castellana' 'Av. Chile' 'Olaya' 'Hospital'
 'Molinos' 'Aguas' 'Museo del Oro' 'Alquería' 'Bosa' 'NQS - Calle 30 Sur'
 'Bicentenario' 'Terreros' 'San Mateo' '

##concatenacion de todos los dataframe

In [ ]:
df_total_list = ["/content/df_total_BMO.csv", "/content/df_total_CAPITALBUS.csv",
                 "/content/df_total_SI18 CALLE 80.csv", "/content/df_total_SI18 NORTE.csv",
                 "/content/df_total_SI18 SUBA.csv", "/content/df_total_SOMOS U.csv"]

df_total = pd.DataFrame()
for csv_file in df_total_list:
  df = pd.read_csv(csv_file)
  df_total = pd.concat([df_total, df], ignore_index=True)
  df_total
df_total= df_total.sort_values(by=["fecha", "NombreParada", "nombre_ruta"], ascending=[True, True, True])
df_total = df_total[df_total["fechaHoraLecturaDato"] >= "03:30:00"] #hay errores el 2023-11-03, se eliminan.

df_total.to_csv("df_total.csv", index=False)

In [ ]:
df_depurada = pd.read_csv("/content/df_total.csv")
result = df_depurada.groupby('fecha')['fechaHoraLecturaDato'].agg(['min', 'max']).reset_index()

print(result)
# se haran 41 tiras por dias correspondientes a 30 minutos

        fecha       min       max
0  2023-10-23  03:59:34  23:59:55
1  2023-10-24  03:32:24  23:59:55
2  2023-10-25  03:56:13  23:59:53
3  2023-10-26  03:53:18  23:59:49
4  2023-10-27  03:53:52  23:59:57
5  2023-10-30  03:58:03  23:59:58
6  2023-10-31  03:55:18  23:59:47
7  2023-11-01  03:53:59  23:59:56
8  2023-11-02  03:46:08  23:59:58
9  2023-11-03  03:40:17  23:59:55


In [ ]:
print(df_depurada["fecha"].unique())

['2023-10-23' '2023-10-24' '2023-10-25' '2023-10-26' '2023-10-27'
 '2023-10-30' '2023-10-31' '2023-11-01' '2023-11-02' '2023-11-03']


In [ ]:
dias = df_depurada["fecha"].unique()

df_depurada['fechaHoraLecturaDato'] = pd.to_datetime(df_depurada['fechaHoraLecturaDato'], format='%H:%M:%S')
df_grafos_temporales = pd.DataFrame(sorted(set(df_depurada["NombreParada"].str.strip())),columns=["NombreParada"])
nom_estaciones = df_grafos_temporales["NombreParada"].unique()

#Se itera por día y por hora para generar cada uno de los grafos
for dia in dias:
  df = {"paradas" : nom_estaciones}
  inicio = datetime.strptime("03:30:00", "%H:%M:%S")
  fin = datetime.strptime("23:59:59", "%H:%M:%S")
  ventana_tiempo = timedelta(hours=0, minutes=30)

  tiempo = inicio

  while tiempo < fin:
    #Filtramos los datos para la ventana actual de tiempo
    current_window_df = df_depurada[
        (df_depurada['fechaHoraLecturaDato'] >= tiempo) &
        (df_depurada['fechaHoraLecturaDato'] < tiempo + ventana_tiempo) &
        (df_depurada['fecha'] == dia)
    ]

    #Agrupamos por nombre de ruta y nombre de parada, simamos ocupaciones
    grouped = current_window_df.groupby(['nombre_ruta', 'NombreParada'])['estimacionOcupacionAbordo'].sum()

    #Iteramos por cada ruta única
    for ruta in df_depurada['nombre_ruta'].unique():
      #Calculamos el respectivo array
      suma_paradas = [grouped.get((ruta, parada), 0) for parada in nom_estaciones]


      df[f"{dia}_{tiempo.strftime('%H:%M:%S')}_{ruta}"] = suma_paradas

      print(f"{dia}_{tiempo.strftime('%H:%M:%S')}_{ruta}")



    tiempo += ventana_tiempo

  #Convertimos el diccionario en un dataframe
  df_time_window = pd.DataFrame(df)

  #Guardamos como csv
  df_time_window.to_csv(f"grafos_temporales_{dia}.csv", index=False)




Streaming output truncated to the last 5000 lines.
2023-11-02_13:30:00_K16 Portal Eldorado
2023-11-02_13:30:00_K23 Portal Dorado
2023-11-02_13:30:00_L18 Portal 20 de Julio
2023-11-02_13:30:00_G41 San Mateo C.C Unisur
2023-11-02_13:30:00_G52 Portal Sur
2023-11-02_13:30:00_L41 Bicentenario
2023-11-02_13:30:00_K10 Portal Eldorado
2023-11-02_13:30:00_L10 Portal 20 de Julio
2023-11-02_13:30:00_6 Portal 80
2023-11-02_13:30:00_6 Universidades
2023-11-02_13:30:00_A61 Calle 72
2023-11-02_13:30:00_B13 Portal Norte
2023-11-02_13:30:00_D24 Portal 80
2023-11-02_13:30:00_F61 Portal Américas
2023-11-02_13:30:00_H13 Portal Tunal
2023-11-02_13:30:00_J24 Universidades
2023-11-02_13:30:00_D20 Portal 80
2023-11-02_13:30:00_D21 Portal 80
2023-11-02_13:30:00_H20 Portal Usme
2023-11-02_13:30:00_H21 Portal Tunal
2023-11-02_13:30:00_E32 NQS Calle 75 Zona M
2023-11-02_13:30:00_F32 Portal Américas
2023-11-02_13:30:00_F51 Portal Américas
2023-11-02_13:30:00_M51 Museo Nacional
2023-11-02_13:30:00_B12 Portal Norte


#Construcción del grafo

In [ ]:
import pandas as pd
import numpy as np
import h5py

In [ ]:
#df_depurada['fechaHoraLecturaDato'] = pd.to_datetime(df_depurada['fechaHoraLecturaDato'], format='%H:%M:%S')

df = pd.read_csv("/content/df_total.csv")
paradas = df["NombreParada"].str.strip().unique()
rutas = df["nombre_ruta"].unique()
print(f"Numero paradas: {len(paradas)}")
print(f"Numero rutas: {len(rutas)}")

#Se calculará el la parada con más rutas para conocer la dimensión a la que se debe reducir la dimensión de los nodos

paradas_rutas = np.array([np.array([parada, len(df[df["NombreParada"] == parada]["nombre_ruta"].unique())]) for parada in paradas])
numero_paradas = np.array(paradas_rutas[:,1], dtype="int64")
print(numero_paradas)
print(paradas_rutas[numero_paradas == 22])

#El maximo numero de paradas corresponde a 23 el cual corresponde a la estacion: "Ricaurte"
#print(paradas_rutas)

Numero paradas: 146
Numero rutas: 81
[ 8  5 13  7  6 15  9  6  3 14 15 17  6  8 14  4  2  8  4  4  8  8  4 10
  9 13 14 15  8  8 15 10 15 15 15 17 16 16 14 17 13 12  7  4  2  9  9  6
  7  4  2  7  3  3  4  4  8 10  4 11  9 13  6 11  4 10 10  4  4 14 11  5
  4  5  7  4 16  8  6  8  8  4 10  4  5  7  8 13  4  7  4  7  6 10  4  6
 13  7 10 11 12 10 17 15 15 11 10  4 15  4  9  6  4  8  8 10 22  6  9  6
  4  2  5  7  2  9  8  6 11  4  8  6 10 11 12  8  7 17  4 10 11  5 10 13
  6  9]
[['Ricaurte' '22']]


In [ ]:
#permite verificar que se tengan las mismas estaciones para cada ruta
for ruta in rutas:
  ruta_real = set(df[df["nombre_ruta"] == ruta]["NombreParada"].unique())
  ruta_consignada = set(dicc_rutas[ruta])
  if ruta_real != ruta_consignada:
    print(f"OJO CON RUTA {ruta}")
    print(set(ruta_real - ruta_real))

81


In [ ]:
#se presenta a continuacion la estructura del grafo, esta tendra la siguiente forma:
# cada elemento correspondera a una tripla ordenada (a,b,c) donde a representa la estacion de partida,
# b la estacion de llegada y c el identificador de la ruta.

#Primero se creara un diccionario con todas las rutas y sus estaciones
dicc_rutas = {
    '4 Héroes - Gelhada' : ['Portal Sur', 'Perdomo', 'Madelena', 'Sevillana',
                            'Venecia', 'Alquería', 'G. Santander', 'NQS - Calle 38A Sur',
                            'NQS - Calle 30 Sur', 'Sena', 'Santa Isabel', 'Comuneros',
                            'Ricaurte', 'Paloquemao', 'CAD', 'Av. El Dorado', 'U. Nacional',
                            'Campín-UAN', 'Movistar Arena', 'Simón Bolívar', 'Av. Chile',
                            'NQS - Calle 75', 'Escuela Militar','Polo',  'Héroes'],

    'G11 Portal Sur' : ['Terminal', 'Calle 187', 'Toberín', 'Mazurén', 'Calle 146',
                        'Pepe Sierra', 'Calle 106', 'Virrey', 'Calle 85', 'Héroes',
                        'Escuela Militar', 'Campín-UAN', 'Paloquemao', 'Santa Isabel','Sena',
                         'NQS - Calle 30 Sur',  'NQS - Calle 38A Sur', 'Alquería',
                        'Venecia', 'Madelena', 'Portal Sur'],

    'G22 Portal Sur JFK' : ['Portal 80', 'Granja - Cra. 77', 'Minuto de Dios',
                            'Av. 68', 'Cra. 47', 'NQS - Calle 75', 'Av. Chile',
                            'U. Nacional', 'Av. El Dorado', 'Paloquemao',
                            'Sena', 'G. Santander', 'Sevillana', 'Portal Sur'],

    'K43 CAD' : ['San Mateo', 'Terreros' , 'León XIII', 'La Despensa',
                 'Venecia', 'G. Santander', 'Sena', 'Ricaurte', 'CAD'],

    'G43 San Mateo C.C. Unisur' : ['Portal Eldorado', 'Modelia', 'Normandía',
                                   'El Tiempo - Maloka', 'Salitre - El Greco',
                                   'Gobernación', 'Quinta Paredes', 'Recinto Ferial',
                                   'CAD', 'Ricaurte', 'Sena', 'G. Santander',
                                   'Venecia', 'La Despensa', 'León XIII', 'Terreros',
                                   'San Mateo'],


    'B11 Terminal' : ['Portal Sur', 'Madelena', 'Venecia', 'Alquería', 'NQS - Calle 38A Sur',
                      'NQS - Calle 30 Sur', 'Sena', 'Santa Isabel','Paloquemao', 'Campín-UAN',
                      'Escuela Militar', 'Héroes', 'Calle 85', 'Virrey', 'Calle 106',
                      'Pepe Sierra', 'Calle 146', 'Mazurén', 'Calle 187',
                      'Terminal'],

    'F51 Portal Américas': ['Museo Nacional', 'San Diego', 'Las Nieves', 'Ricaurte',
                            'Distrito Grafiti', 'Pradera', 'Marsella',
                            'Av. Américas - Av. Boyacá', 'Mandalay', 'Banderas',
                            'Patio Bonito', 'Portal Américas'],



    '4 Portal Sur JFK' : ['Héroes', 'Polo', 'Escuela Militar', 'NQS - Calle 75',
                          'Av. Chile', 'Simón Bolívar', 'Movistar Arena', 'Campín-UAN',
                          'U. Nacional', 'Av. El Dorado', 'CAD', 'Paloquemao', 'Ricaurte',
                          'Comuneros', 'Santa Isabel', 'Sena', 'NQS - Calle 30 Sur',
                          'NQS - Calle 38A Sur', 'G. Santander', 'Alquería', 'Venecia',
                          'Sevillana', 'Madelena', 'Perdomo', 'Portal Sur'],



    'K43 Portal ElDorado': ['San Mateo', 'Terreros', 'León XIII', 'La Despensa',
                            'Venecia', 'G. Santander', 'Sena', 'Ricaurte', 'CAD',
                            'Recinto Ferial', 'Quinta Paredes', 'Gobernación',
                             'Salitre - El Greco', 'El Tiempo - Maloka', 'Normandía',
                            'Modelia' , 'Portal Eldorado'],

    'D22 Portal 80': ['Portal Sur', 'Sevillana', 'G. Santander', 'Sena', 'Paloquemao',
                      'Av. El Dorado', 'U. Nacional', 'Av. Chile', 'NQS - Calle 75',
                      'Cra. 47', 'Av. 68', 'Minuto de Dios', 'Granja - Cra. 77',
                      'Portal 80'],

    '7 Portal Suba' : ['Santa Isabel','Comuneros','Ricaurte', 'Paloquemao', 'CAD',
                       'Av. El Dorado','U. Nacional', 'Campín-UAN', 'Movistar Arena',
                       'Simón Bolívar', 'Av. Chile', 'NQS - Calle 75',
                       'San Martín', 'Rionegro',  'Suba - Calle 95', 'Suba - Calle 100',
                       'Puentelargo', 'Av. Suba - Calle 116', 'Humedal Córdoba', 'Niza - Calle 127',
                       'Suba - Av. Boyacá', 'Gratamira', '21 Ángeles', 'Suba - Tv. 91', 'La Campiña', 'Portal Suba'],
    '7 Santa Isabel' : ['Portal Suba', 'La Campiña', 'Suba - Tv. 91', '21 Ángeles', 'Gratamira',
                        'Suba - Av. Boyacá', 'Niza - Calle 127', 'Humedal Córdoba',
                        'Av. Suba - Calle 116', 'Puentelargo', 'Suba - Calle 100',
                        'Suba - Calle 95', 'Rionegro', 'San Martín', 'NQS - Calle 75',
                        'Av. Chile', 'Simón Bolívar', 'Movistar Arena', 'Campín-UAN',
                        'U. Nacional', 'Av. El Dorado', 'CAD', 'Paloquemao', 'Ricaurte',
                        'Comuneros', 'Santa Isabel'],
    'B50 Calle 161' : ['Portal Suba', 'La Campiña', 'Suba - Tv. 91', '21 Ángeles',
                         'Av. Suba - Calle 116' , 'Puentelargo', 'Rionegro',
                         'Héroes', 'Calle 85', 'Virrey', 'Calle 142', 'Calle 161'],
    'C17 Portal Suba' : ['Portal Usme', 'Consuelo', 'Santa Lucía', 'Quiroga',
                         'Restrepo', 'Hortúa', 'Av. 39', 'Calle 57', 'Calle 76',
                         'Polo', 'San Martín', 'Suba - Calle 95', 'Suba - Calle 100',
                         'Av. Suba - Calle 116', '21 Ángeles', 'La Campiña',
                         'Portal Suba'],
    'C25 Portal Suba' : ['PORTAL 20 DE JULIO', 'Country Sur', 'Av. 1º de Mayo',
                         'San Bernardo','Bicentenario', 'Tygua - San José', 'Guatoque - Veraguas',
                         'Campín-UAN', 'Movistar Arena', 'Av. Chile', 'Suba - Calle 95',
                         'Niza - Calle 127', 'Suba - Av. Boyacá',
                         '21 Ángeles','Portal Suba'],
    'C50 Portal Suba' : ['Calle 161', 'Calle 142', 'Virrey', 'Calle 85', 'Héroes',
                         'Rionegro', 'Puentelargo', 'Av. Suba - Calle 116',
                         '21 Ángeles', 'Suba - Tv. 91', 'La Campiña', 'Portal Suba'],
    'H17 Portal Usme' : ['Portal Suba', 'La Campiña', '21 Ángeles', 'Av. Suba - Calle 116',
                         'Suba - Calle 100', 'Suba - Calle 95', 'San Martín',
                         'Polo', 'Calle 76', 'Calle 57', 'Av. 39', 'Hortúa',
                         'Restrepo', 'Quiroga', 'Santa Lucía', 'Consuelo', 'Portal Usme'],
    'L25 Portal 20 de Julio' : ['Portal Suba', '21 Ángeles', 'Suba - Av. Boyacá',
                                'Niza - Calle 127', 'Suba - Calle 95', 'Av. Chile',
                                'Movistar Arena', 'Campín-UAN', 'Guatoque - Veraguas',
                                'Tygua - San José', 'Bicentenario', 'San Bernardo',
                                'Av. 1º de Mayo', 'Country Sur', 'PORTAL 20 DE JULIO'],
    'B74 Portal Norte' : ['Aguas', 'Museo del Oro',  'Calle 19', 'Calle 26',
                          'Av. 39', 'Calle 45', 'Calle 57', 'Héroes', 'Virrey',
                          'Calle 127', 'Prado', 'Calle 142', 'Calle 146', 'Toberín', 'Portal Norte'],
    'F23 Portal Américas' : ['Aguas', 'Museo del Oro', 'Av. Jiménez',
                             'San Façon - Cra. 22', 'Ricaurte', 'Puente Aranda',
                             'Pradera', 'Av. Américas - Av. Boyacá', 'Banderas',
                             'Patio Bonito', 'Portal Américas'],
    'J23 Las Aguas' : ['Portal Américas', 'Patio Bonito', 'Banderas',
                       'Av. Américas - Av. Boyacá', 'Pradera', 'Puente Aranda',
                       'Ricaurte', 'San Façon - Cra. 22', 'Av. Jiménez',
                       'Museo del Oro', 'Aguas'],
    'J70 Las Aguas' : [ 'Portal Norte', 'Toberín','Calle 146', 'Calle 142',
                       'Alcalá', 'Prado', 'Calle 127', 'CAD', 'Av. Jiménez',
                        'Museo del Oro', 'Aguas'],
    'J74 Las Aguas' : ['Portal Norte', 'Toberín', 'Calle 146', 'Calle 142',
                       'Prado', 'Calle 127', 'Virrey', 'Héroes',
                       'Calle 57', 'Calle 45', 'Av. 39', 'Calle 26',
                       'Calle 19', 'Museo del Oro', 'Aguas'],
    '8 Estación Guatoque Veraguas' : ['Terminal', 'Calle 187', 'Portal Norte',
                                       'Toberín', 'Calle 161', 'Mazurén',
                                       'Calle 146', 'Calle 142', 'Alcalá',
                                       'Prado', 'Calle 127', 'Pepe Sierra',
                                       'Calle 106', 'Calle 100', 'Virrey',
                                       'Calle 85', 'Héroes', 'Calle 76',
                                       'Calle 72', 'Flores', 'Calle 63',
                                       'Calle 57', 'Marly', 'Calle 45',
                                       'Av. 39', 'Calle 34', 'Calle 26', 'Calle 22',
                                       'Calle 19', 'Av. Jiménez', 'Tercer Milenio',
                                      'Tygua - San José','Guatoque - Veraguas'],
    '8 Estación Terminal' : ['Guatoque - Veraguas', 'Tygua - San José', 'Tercer Milenio',
                             'Av. Jiménez', 'Calle 19', 'Calle 22', 'Calle 26',
                             'Calle 34', 'Av. 39', 'Calle 45', 'Marly', 'Calle 57',
                             'Calle 63', 'Flores', 'Calle 72', 'Calle 76', 'Héroes',
                             'Calle 85', 'Virrey', 'Calle 100', 'Calle 106', 'Pepe Sierra',
                             'Calle 127', 'Prado', 'Alcalá', 'Calle 142', 'Calle 146',
                             'Mazurén', 'Calle 161', 'Toberín', 'Portal Norte',
                             'Calle 187', 'Terminal'],
    'B10 Portal Norte' : ['Portal 80', 'Cra. 90',  'Av. Cali', 'Granja - Cra. 77',
                          'Av. 68', 'Cra. 53', 'Cra. 47', 'Calle 85', 'Virrey',
                          'Calle 100' , 'Calle 106', 'Pepe Sierra', 'Alcalá',
                          'Calle 146', 'Toberín', 'Portal Norte'],
    'B16 Terminal' : ['Portal Eldorado', 'Modelia', 'Av. Rojas', 'El Tiempo - Maloka',
                      'CAN', 'Quinta Paredes', 'Recinto Ferial',  'Av. El Dorado',
                      'Av. Chile', 'Pepe Sierra', 'Alcalá', 'Toberín', 'Calle 187',
                      'Terminal'],
    'B18 Terminal' : ['PORTAL 20 DE JULIO', 'Country Sur', 'Av. 1º de Mayo',
                      'San Bernardo', 'Bicentenario', 'Tercer Milenio', 'Calle 34',
                      'Calle 45', 'Calle 63', 'Virrey', 'Pepe Sierra', 'Calle 127',
                      'Alcalá', 'Calle 187', 'Terminal'],
    'B23 Alcalá' : ['Portal Eldorado', 'Av. Rojas', 'El Tiempo - Maloka',
                    'Salitre - El Greco', 'CAN', 'Gobernación',
                    'Ciudad Universitaria', 'Concejo de Bogotá', 'Calle 26',
                    'Calle 45', 'Marly','Calle 57', 'Calle 85', 'Calle 127', 'Prado',
                    'Alcalá'],
    'B72 Toberín' : ['Portal Usme', 'Molinos', 'Socorro', 'Olaya', 'Fucha', 'Nariño',
                     'Hospital', 'Tygua - San José', 'Paloquemao', 'Movistar Arena', 'Av. Chile',
                     'Castellana', 'Pepe Sierra', 'Alcalá', 'Toberín'],
    'H72 Portal Usme' : ['Toberín', 'Alcalá', 'Pepe Sierra', 'Castellana',
                         'Av. Chile', 'Movistar Arena', 'Paloquemao',
                         'Tygua - San José', 'Hospital', 'Nariño', 'Fucha',
                         'Olaya', 'Socorro', "Consuelo", 'Portal Usme'],
    'H73 Portal Tunal' : ['Toberín', 'Calle 146', 'Alcalá', 'Prado', 'Calle 127',
                          'Calle 45', 'Av. 39', 'Calle 34', 'Calle 26', 'Calle 19',
                          'Fucha', 'Olaya',  'Calle 40 S.', 'Portal Tunal'],
    'K16 Portal Eldorado' : ['Terminal', 'Calle 187', 'Toberín', 'Alcalá',
                             'Pepe Sierra', 'Av. Chile', 'Av. El Dorado',
                             'Recinto Ferial', 'Quinta Paredes', 'CAN', 'Salitre - El Greco',
                             'El Tiempo - Maloka', 'Av. Rojas', 'Modelia', 'Portal Eldorado'],
    'K23 Portal Dorado' : ['Alcalá', 'Prado', 'Calle 127', 'Calle 85', 'Calle 57',
                           'Marly', 'Calle 45', 'Calle 26', 'Concejo de Bogotá',
                           'Ciudad Universitaria', 'Gobernación', 'CAN', 'Salitre - El Greco',
                           'El Tiempo - Maloka', 'Av. Rojas', 'Portal Eldorado'],
    'L18 Portal 20 de Julio' : ['Terminal', 'Calle 187', 'Alcalá', 'Calle 127',
                                'Pepe Sierra', 'Virrey', 'Calle 63', 'Calle 45',
                                'Calle 34', 'Tercer Milenio', 'Bicentenario',
                                'San Bernardo', 'Av. 1º de Mayo', 'Country Sur',
                                'PORTAL 20 DE JULIO'],
    'G41 San Mateo C.C Unisur' : ['Bicentenario', 'Tygua - San José',
                                  'Guatoque - Veraguas', 'NQS - Calle 30 Sur',
                                  'Alquería', 'Bosa', 'Terreros','San Mateo'],
    'G52 Portal Sur' : ['Flores', 'Calle 76', 'Santa Isabel', 'NQS - Calle 30 Sur',
                        'G. Santander', 'Alquería','Venecia', 'Madelena','Perdomo', 'Portal Sur'],
    'L41 Bicentenario' : ['San Mateo', 'Terreros', 'Bosa', 'Alquería',
                          'NQS - Calle 30 Sur', 'Guatoque - Veraguas', 'Tygua - San José',
                          'Bicentenario'],
    'K10 Portal Eldorado' : ['PORTAL 20 DE JULIO', 'Av. 1º de Mayo', 'Ciudad Jardín-UAN',
                             'San Bernardo', 'Bicentenario','Tercer Milenio', 'Av. Jiménez',
                             'Calle 22', 'Centro Memoria',  'Ciudad Universitaria', 'Gobernación',
                             'CAN', 'Salitre - El Greco', 'Normandía', 'Modelia',
                             'Portal Eldorado'],
    'L10 Portal 20 de Julio' : ['Portal Eldorado', 'Modelia', 'Normandía',
                                'Salitre - El Greco', 'CAN', 'Gobernación',
                                'Ciudad Universitaria', 'Centro Memoria', 'Calle 22',
                                'Av. Jiménez', 'Tercer Milenio', 'Bicentenario',
                                'San Bernardo', 'Ciudad Jardín-UAN', 'Av. 1º de Mayo',
                                'PORTAL 20 DE JULIO'],
    '6 Portal 80' : ['Universidades','Calle 26','Calle 34', 'Av. 39',  'Calle 45',  'Marly' ,
                     'Calle 57', 'Calle 63', 'Flores', 'Calle 72', 'Calle 76',
                     'Polo', 'Escuela Militar', 'Cra. 47', 'Cra. 53',
                      'Av. 68', 'Ferias', 'Boyacá', 'Minuto de Dios' , 'Granja - Cra. 77',
                     'Quirigua', 'Portal 80'],
    '6 Universidades' : ['Portal 80', 'Quirigua', 'Granja - Cra. 77', 'Minuto de Dios',
                         'Boyacá', 'Ferias', 'Av. 68', 'Cra. 53', 'Cra. 47',
                         'Escuela Militar', 'Polo', 'Calle 76', 'Calle 72', 'Flores',
                         'Calle 63', 'Calle 57', 'Marly', 'Calle 45', 'Av. 39', 'Calle 34',
                         'Calle 26', 'Universidades'],
    'A61 Calle 72' : ['Portal Américas', 'Patio Bonito', 'Banderas', 'Marsella',
                      'CDS - Cra. 32', 'Calle 19', 'Av. 39', 'Calle 57', 'Flores',
                      'Calle 72'],
    'B13 Portal Norte' : ['Portal Tunal', 'Biblioteca','Calle 40 S.', 'Quiroga', 'Restrepo',
                          'Hortúa', 'Tercer Milenio', 'Calle 22', 'Av. 39',
                          'Calle 63' , 'Flores', 'Calle 72', 'Calle 85', 'Calle 106', 'Calle 142',
                          'Calle 146', 'Mazurén', 'Calle 161','Portal Norte'],
    'D24 Portal 80' : ['Universidades', 'Av. 39', 'Marly', 'Calle 57', 'Calle 72', 'Calle 76',
                       'Polo', 'Escuela Militar','Cra. 47', 'Minuto de Dios' ,
                       'Granja - Cra. 77', 'Portal 80'],
    'F61 Portal Américas' : ['Calle 72', 'Flores', 'Calle 57', 'Av. 39', 'Calle 19',
                             'CDS - Cra. 32', 'Marsella', 'Banderas', 'Patio Bonito',
                             'Portal Américas'],
    'H13 Portal Tunal' : ['Portal Norte', 'Calle 161', 'Mazurén', 'Calle 146',
                          'Calle 142', 'Calle 106', 'Calle 85', 'Calle 72', 'Flores',
                          'Calle 63', 'Av. 39', 'Calle 22', 'Tercer Milenio', 'Hortúa',
                          'Restrepo', 'Quiroga', 'Calle 40 S.', 'Biblioteca', 'Portal Tunal'],
    'J24 Universidades' : ['Portal 80', 'Granja - Cra. 77', 'Minuto de Dios',
                           'Cra. 47', 'Escuela Militar', 'Polo', 'Calle 76',
                           'Calle 72', 'Calle 57', 'Marly', 'Av. 39', 'Universidades'],
    'D20 Portal 80' : ['Portal Usme', 'Molinos', 'Calle 40 S.', 'Quiroga', 'Restrepo', 'Av. Jiménez',
                       'Calle 19','Calle 26', 'Calle 34', 'Calle 45', 'Calle 63',
                        'Flores', 'Polo', 'Av. 68', 'Ferias', 'Boyacá', 'Granja - Cra. 77',
                       'Quirigua','Portal 80'],
    'D21 Portal 80' : ['Portal Tunal', 'Santa Lucía', 'Calle 40 S.', 'Fucha',
                       'Guatoque - Veraguas', 'Ricaurte',
                       'Campín-UAN', 'Movistar Arena', 'Simón Bolívar', 'Av. 68', 'Minuto de Dios',
                       'Granja - Cra. 77','Av. Cali', 'Cra. 90', 'Portal 80', ],
    'H20 Portal Usme' : ['Portal 80', 'Quirigua', 'Granja - Cra. 77', 'Boyacá',
                         'Ferias', 'Av. 68', 'Polo', 'Flores', 'Calle 63', 'Calle 45',
                         'Calle 34', 'Calle 26', 'Calle 19', 'Av. Jiménez', 'Restrepo','Quiroga',
                         'Calle 40 S.', 'Consuelo', 'Portal Usme'],
    'H21 Portal Tunal' : ['Portal 80', 'Cra. 90', 'Av. Cali', 'Granja - Cra. 77',
                          'Minuto de Dios', 'Av. 68', 'Simón Bolívar','Movistar Arena', 'Campín-UAN',
                          'Ricaurte', 'Guatoque - Veraguas', 'Fucha', 'Calle 40 S.',
                          'Santa Lucía', 'Portal Tunal'],
    'E32 NQS Calle 75 Zona M' : ['Portal Américas', 'Tv. 86', 'Banderas',
                                 'Av. Américas - Av. Boyacá', 'Puente Aranda',
                                  'Zona Industrial', 'Av. El Dorado',  'U. Nacional',
                                 'Simón Bolívar', 'Av. Chile', 'NQS - Calle 75'],
    'F32 Portal Américas' :['NQS - Calle 75', 'Av. Chile', 'Simón Bolívar', 'U. Nacional',
                            'Av. El Dorado', 'Zona Industrial', 'Puente Aranda',
                            'Av. Américas - Av. Boyacá', 'Banderas', 'Tv. 86',
                            'Portal Américas'],
    'M51 Museo Nacional' : ['Portal Américas', 'Patio Bonito', 'Banderas',
                            'Mandalay', 'Av. Américas - Av. Boyacá',
                            'Marsella', 'Pradera', 'Distrito Grafiti',
                            'Ricaurte','Las Nieves', 'San Diego', 'Museo Nacional'],
    'B12 Portal Norte' : ['Portal Sur', 'Perdomo', 'G. Santander', 'Comuneros',
                          'Ricaurte', 'Av. El Dorado', 'U. Nacional', 'Simón Bolívar',
                          'NQS - Calle 75', 'Castellana', 'Calle 100', 'Calle 127',
                          'Prado', 'Calle 142', 'Toberín', 'Portal Norte'],
    'C30 Portal Suba' : ['Portal Sur', 'Venecia',  'G. Santander', 'NQS - Calle 38A Sur',
                         'Santa Isabel','Ricaurte', 'Av. El Dorado','Campín-UAN',
                         'NQS - Calle 75', 'Suba - Calle 95', 'Puentelargo',
                         'Av. Suba - Calle 116', 'Suba - Av. Boyacá', 'La Campiña',
                         'Suba - Tv. 91','Portal Suba'],
    'G12 Gral. Santander' : ['Portal Norte', 'Toberín', 'Calle 142', 'Prado', 'Calle 127',
                             'Calle 100', 'Castellana', 'NQS - Calle 75',
                             'Simón Bolívar', 'U. Nacional', 'Av. El Dorado',
                             'Ricaurte', 'Comuneros', 'G. Santander'],
    'G12 Portal Sur' : ['Portal Norte', 'Toberín', 'Calle 142', 'Prado', 'Calle 127',
                        'Calle 100', 'Castellana', 'NQS - Calle 75', 'Simón Bolívar',
                        'U. Nacional', 'Av. El Dorado', 'Ricaurte', 'Comuneros',
                        'G. Santander', 'Perdomo', 'Portal Sur'],
    'G30 Portal Sur JFK' : ['Portal Suba', 'Suba - Tv. 91', 'La Campiña', 'Suba - Av. Boyacá',
                            'Av. Suba - Calle 116', 'Puentelargo', 'Suba - Calle 95',
                            'NQS - Calle 75', 'Campín-UAN', 'Av. El Dorado',
                            'Ricaurte', 'Santa Isabel', 'NQS - Calle 38A Sur',
                            'G. Santander', 'Venecia', 'Portal Sur'],
    '3 Corferias' : ['Portal Tunal', 'Parque', 'Biblioteca', 'Santa Lucía',
                     'Calle 40 S.', 'Quiroga',  'Olaya', 'Restrepo', 'Fucha',
                     'Nariño', 'Hortúa', 'Hospital', 'Tercer Milenio',
                     'Av. Jiménez',  'Calle 19', 'Calle 22', 'Centro Memoria',
                     'Concejo de Bogotá', 'Ciudad Universitaria' , 'Recinto Ferial'],
    '3 Portal Tunal' : ['Recinto Ferial', 'Ciudad Universitaria', 'Concejo de Bogotá',
                        'Centro Memoria', 'Calle 22', 'Calle 19', 'Av. Jiménez',
                        'Tercer Milenio', 'Hospital', 'Hortúa', 'Nariño', 'Fucha',
                        'Restrepo', 'Olaya', 'Quiroga', 'Calle 40 S.', 'Santa Lucía',
                        'Biblioteca', 'Parque', 'Portal Tunal'],
    'B27 Portal Norte ' : ['Portal Tunal', 'Calle 40 S.', 'Av. Jiménez',
                           'Calle 34','Calle 76', 'Héroes', 'Calle 85', 'Virrey',
                           'Calle 100','Toberín', 'Portal Norte'],
    'B75 Portal Norte' : [ 'Portal Usme', 'Molinos',  'Consuelo', 'Socorro',
                          'Santa Lucía', 'Fucha', 'Hortúa', 'Av. Jiménez',
                           'Marly' , 'Calle 63', 'Calle 72', 'Calle 76',
                           'Héroes', 'Calle 100', 'Prado', 'Calle 142',
                           'Calle 146', 'Toberín', 'Portal Norte'],
    'C15 Portal Suba': ['Portal Tunal', 'Parque', 'Calle 40 S.', 'Olaya' ,
                        'Tercer Milenio', 'Av. Jiménez', 'Calle 19', 'Calle 22',
                        'Calle 45', 'Marly', 'Calle 57', 'Calle 76', 'Escuela Militar',
                        'Rionegro', 'Puentelargo', 'Humedal Córdoba',
                        'Niza - Calle 127', 'Gratamira', 'Suba - Tv. 91',
                        'Portal Suba'],
    'H15 Portal Suba' : ['Portal Suba', 'Suba - Tv. 91', 'Gratamira', 'Niza - Calle 127',
                          'Humedal Córdoba', 'Puentelargo', 'Rionegro', 'Escuela Militar',
                          'Calle 76', 'Calle 57', 'Marly', 'Calle 45', 'Calle 22',
                          'Calle 19', 'Av. Jiménez', 'Tercer Milenio', 'Olaya',
                          'Calle 40 S.', 'Parque', 'Portal Tunal'],
    'H27 Portal Tunal' : ['Portal Norte', 'Calle 100', 'Virrey',
                          'Calle 85', 'Héroes', 'Calle 76', 'Calle 34', 'Av. Jiménez',
                          'Calle 40 S.', 'Portal Tunal'],
    'H75 Portal Usme' : ['Portal Norte', 'Calle 146', 'Calle 142',
                         'Prado', 'Calle 100', 'Héroes', 'Calle 76', 'Calle 72',
                         'Calle 63', 'Marly', 'Av. Jiménez', 'Hortúa', 'Fucha',
                         'Santa Lucía', 'Socorro', 'Consuelo', 'Portal Usme'],
    'H54 Portal Usme' : ['Portal Eldorado', 'Modelia', 'Av. Rojas', 'El Tiempo - Maloka',
                         'Salitre - El Greco', 'CAN', 'Quinta Paredes', 'Recinto Ferial',
                         'Ciudad Universitaria', 'Concejo de Bogotá', 'Hortúa',
                         'Restrepo', 'Calle 40 S.', 'Consuelo', 'Portal Usme'],
    'K54 Portal Eldorado' : ['Portal Usme', 'Molinos', 'Calle 40 S.', 'Restrepo',
                             'Hortúa', 'Concejo de Bogotá', 'Ciudad Universitaria',
                             'Recinto Ferial', 'Quinta Paredes', 'CAN',
                             'Salitre - El Greco', 'El Tiempo - Maloka',
                             'Av. Rojas', 'Modelia', 'Portal Eldorado'],
    'A60 Calle 76' : ['Portal Américas', 'Tv. 86', 'Banderas', 'Cra. 43',
                       'Ricaurte', 'De la Sabana', 'Calle 26', 'Calle 34',
                      'Calle 45', 'Marly', 'Calle 63', 'Calle 76'],
    'B28 Portal Norte' : ['Portal Américas', 'Biblioteca Tintal', 'Banderas',
                          'Marsella', 'Zona Industrial', 'CDS - Cra. 32',
                          'Castellana' , 'Calle 106', 'Pepe Sierra',
                          'Prado', 'Calle 146', 'Calle 161', 'Portal Norte'],
    'C19 Portal Suba' : ['Banderas', 'Mandalay','Marsella', 'Distrito Grafiti',
                         'Zona Industrial', 'Ricaurte', 'De la Sabana',
                         'Calle 19', 'Calle 26', 'Calle 34', 'Marly',
                         'Calle 63', 'Calle 72', 'Suba - Calle 95', 'Suba - Calle 100',
                         'Suba - Av. Boyacá', 'Suba - Tv. 91', 'La Campiña', 'Portal Suba'],
    'F19 Banderas' :['Portal Suba', 'La Campiña', 'Suba - Tv. 91', 'Suba - Av. Boyacá',
                     'Suba - Calle 100', 'Suba - Calle 95', 'Calle 72', 'Calle 63',
                     'Marly', 'Calle 34', 'Calle 26', 'Calle 19', 'De la Sabana',
                     'Ricaurte', 'Zona Industrial', 'Distrito Grafiti',
                     'Marsella', 'Mandalay', 'Banderas'],
    'F28 Portal Américas' : ['Portal Norte', 'Calle 161', 'Calle 146', 'Prado',
                             'Pepe Sierra', 'Calle 106', 'Castellana', 'CDS - Cra. 32',
                             'Zona Industrial', 'Marsella', 'Banderas',
                             'Biblioteca Tintal', 'Portal Américas'],
    'F60 Portal Américas' : ['Calle 76', 'Calle 63', 'Marly', 'Calle 45',
                             'Calle 34', 'Calle 26', 'De la Sabana', 'Ricaurte',
                             'Cra. 43', 'Banderas', 'Tv. 86', 'Portal Américas'],
    'G45 Portal Sur JFK' : ['San Mateo', 'Terreros', 'León XIII',
                            'La Despensa', 'Bosa', 'Portal Sur'],
    'G45 San Mateo C.C Unisur ' : ['Portal Sur', 'Bosa', 'La Despensa',
                                   'León XIII', 'Terreros', 'San Mateo'],
    'J73 Universidades' : ['Portal Suba', 'La Campiña' , 'Suba - Tv. 91',
                           'Niza - Calle 127', 'Puentelargo', 'San Martín' ,
                           'Ciudad Universitaria','Universidades' ],
    'G47 Portal Sur' : ['Museo Nacional', 'San Diego', 'Las Nieves',
                        'San Victorino', 'Tygua - San José', 'Guatoque - Veraguas',
                        'Comuneros', 'NQS - Calle 38A Sur', 'Sevillana',
                        'Madelena','Perdomo', 'Portal Sur' ],
    'M47 Museo Nacional' : ['Portal Sur', 'Perdomo', 'Madelena', 'Sevillana',
                            'NQS - Calle 38A Sur', 'Comuneros', 'Guatoque - Veraguas',
                            'Tygua - San José', 'San Victorino', 'Las Nieves',
                            'San Diego', 'Museo Nacional']}

tuplas_grafo = []

#
for nom_ruta_key, paradas_list in dicc_rutas.items():
  for i, parada_list in enumerate(paradas_list):
    parada_derecha = paradas_list[i + 1] if i + 1 < len(paradas_list) else None
    if None not in (parada_list, parada_derecha, nom_ruta_key):
      g_element = (np.where(paradas == parada_list)[0][0], np.where(paradas == parada_derecha)[0][0], np.where(rutas == nom_ruta_key)[0][0])
      tuplas_grafo.append(g_element)

for tup in tuplas_grafo:
    print(tup)





(104, 97, 72)
(97, 74, 72)
(74, 127, 72)
(127, 142, 72)
(142, 3, 72)
(3, 61, 72)
(61, 86, 72)
(86, 85, 72)
(85, 145, 72)
(145, 125, 72)
(125, 46, 72)
(46, 116, 72)
(116, 93, 72)
(93, 20, 72)
(20, 10, 72)
(10, 140, 72)
(140, 41, 72)
(41, 82, 72)
(82, 128, 72)
(128, 9, 72)
(9, 87, 72)
(87, 57, 72)
(57, 98, 72)
(98, 69, 72)
(135, 29, 75)
(29, 137, 75)
(137, 78, 75)
(78, 27, 75)
(27, 96, 75)
(96, 24, 75)
(24, 143, 75)
(143, 40, 75)
(40, 69, 75)
(69, 57, 75)
(57, 41, 75)
(41, 93, 75)
(93, 125, 75)
(125, 145, 75)
(145, 85, 75)
(85, 86, 75)
(86, 3, 75)
(3, 142, 75)
(142, 74, 75)
(74, 104, 75)
(99, 63, 77)
(63, 79, 77)
(79, 6, 77)
(6, 51, 77)
(51, 87, 77)
(87, 9, 77)
(9, 140, 77)
(140, 10, 77)
(10, 93, 77)
(93, 145, 77)
(145, 61, 77)
(61, 127, 77)
(127, 104, 77)
(123, 136, 79)
(136, 73, 79)
(73, 71, 79)
(71, 142, 79)
(142, 61, 79)
(61, 145, 79)
(145, 116, 79)
(116, 20, 79)
(101, 80, 78)
(80, 90, 78)
(90, 56, 78)
(56, 118, 78)
(118, 62, 78)
(62, 111, 78)
(111, 114, 78)
(114, 20, 78)
(20, 116, 7

In [ ]:
# Construcción del grafo (agregar colores para mostrar lineas del tm)
gml_filename = "grafo_transmilenio.gml"

color_map = {
    "#00ff00" : [2,23,24,25,26,27,28,29,40,69,78,96,102,108,128,135,137,144], #verde claro B
    "#ffff00" : [0,13,64,68,70,89,103,110,117,122,130,131,132,133,], #amarillo c
    "#996600" : [9,10,20,41,42,65,82,87,93,139,140,], #cafe claro zona E
    "#9966ff" : [6,8,19,51,52,53,57,58, 63,79,98,99, 112,], #morado zona D
    "#0000ff" : [5,30,31,32,33,35,36,37,38,39,59,76,134,], #azul zona A
    "#ff0000" : [7,14,16,22,50, 54, 55,75,77,95,100,107, 109,116,121,138,145], #rojo zona F
    "#0099ff" : [3,18,44,46,61,71,73,74,85,86,97,104,123,125,126,127,136,143], #azul claro zona G
    "#ff9900" : [15,34,48,60,66,67,81,88,91,94,105,106,113,115,129], #naranja zona H
    "#ff99ff" : [1,11, 84,142,], #rozado zona j
    "#cccc00" : [12,21,43,45,47,56, 62, 80,90,101,111,114,118], #color feo zona k
    "#006699" : [4,17,49,72,92,119, 120,124,], #azul metalizado zona L
    "#660066" : [83], #morado zona M
    }

with open(gml_filename, "w") as gml_file:
  gml_file.write("graph [\n")
  gml_file.write("  directed 1\n")

  nodes = set()
  for edge in tuplas_grafo:
    nodes.add(edge[0])
    nodes.add(edge[1])

  for node in nodes:
    node_color = None
    for color, node_list in color_map.items():
      if node in node_list:
        node_color = color
        break


    gml_file.write("  node [\n")
    gml_file.write(f"    id {node}\n")
    gml_file.write(f"    label \"{paradas[node]}\"\n")
    if node_color:
      gml_file.write(f"    graphics [\n      fill \"{node_color}\"\n    ]\n")
    gml_file.write("  ]\n")

  for origen, destino, _ in tuplas_grafo:
    gml_file.write(
      f"  edge [\n"
      f"    source {origen}\n"
      f"    target {destino}\n"
      f"  ]\n")

  gml_file.write("]\n")





In [ ]:
df_23 = pd.read_csv("/content/grafos_temporales_2023-10-23.csv")
df_24 = pd.read_csv("/content/grafos_temporales_2023-10-24.csv")
df_25= pd.read_csv("/content/grafos_temporales_2023-10-25.csv")
df_26 = pd.read_csv("/content/grafos_temporales_2023-10-26.csv")
df_27 = pd.read_csv("/content/grafos_temporales_2023-10-27.csv")

df_30 = pd.read_csv("/content/grafos_temporales_2023-10-30.csv")
df_31 = pd.read_csv("/content/grafos_temporales_2023-10-31.csv")
df_01 = pd.read_csv("/content/grafos_temporales_2023-11-01.csv")
df_02 = pd.read_csv("/content/grafos_temporales_2023-11-02.csv")
df_03 = pd.read_csv("/content/grafos_temporales_2023-11-03.csv")


periodo = 81 #numero de rutas
df_intento = pd.concat([df_23.iloc[:,1:], df_24.iloc[:,1:], df_25.iloc[:,1:], df_26.iloc[:,1:],
                        df_27.iloc[:,1:], df_30.iloc[:,1:], df_31.iloc[:,1:], df_01.iloc[:,1:],
                        df_02.iloc[:,1:], df_03.iloc[:,1:]],axis = 1)
dicc_matrices_adyacencia = {}
for i in range(410): #numero total de ventanas de tiempo 41*10
  df_static_graph = df_intento.iloc[:, i*periodo:(i+1)*periodo]
  matriz_adyacencia = np.zeros((len(paradas), len(paradas)))
  for tupla in tuplas_grafo:
    matriz_adyacencia[tupla[0], tupla[1]] += ((df_static_graph.iloc[tupla[1],tupla[2]]) - (df_static_graph.iloc[tupla[0],tupla[2]]))
  dicc_matrices_adyacencia[i] = matriz_adyacencia

with h5py.File('matrices_adyacencia.h5', 'w') as f:
  for key, matrix in dicc_matrices_adyacencia.items():
    f.create_dataset(str(key), data=matrix)

In [ ]:
with h5py.File('matrices_adyacencia.h5', 'r') as f:
  loaded_dict = {key: f[key][...] for key in f.keys()}

print(loaded_dict)

{'0': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '1': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '10': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '100': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '101': a

#Masking y obtencion de caracteristicas temporales

In [ ]:
#matriz de masking
#la idea es para cada estacion extraer solo las rutas con las que interactua,
#se determinó que el número de rutas máximo que puede tener una estación
#corresponde a 17, por lo tanto, se creará un diccionario que indique las celdas
#con las que interacciona

"""
dicc_masking = {}
for i, _ in enumerate(paradas):
  set_rutas = set()
  for tupl in tuplas_grafo:
    if i in tupl[:2]:
      set_rutas.add(tupl[2])

  dicc_masking[i] = list(set_rutas)

list_len = []
for key, value in dicc_masking.items():
  list_len.append(len(value))
ESTO FUE UTILIZADO PARA LA SHALLOW"""




[8, 5, 13, 7, 6, 15, 9, 6, 3, 14, 15, 17, 6, 8, 14, 4, 2, 8, 4, 4, 8, 8, 4, 10, 9, 13, 14, 15, 8, 8, 15, 10, 15, 15, 15, 17, 16, 16, 14, 17, 13, 12, 7, 4, 2, 9, 9, 6, 7, 4, 2, 7, 3, 3, 4, 4, 8, 10, 4, 11, 9, 13, 6, 11, 4, 10, 10, 4, 4, 14, 11, 5, 4, 5, 7, 4, 16, 8, 6, 8, 8, 4, 10, 4, 5, 7, 8, 13, 4, 7, 4, 7, 6, 10, 4, 6, 13, 7, 10, 11, 12, 10, 17, 15, 15, 11, 10, 4, 15, 4, 9, 6, 4, 8, 8, 10, 22, 6, 9, 6, 4, 2, 5, 7, 2, 9, 8, 6, 11, 4, 8, 6, 10, 11, 12, 8, 7, 17, 4, 10, 11, 5, 10, 13, 6, 9]


In [ ]:
#estandarizacion de cada columna para input de la red
from sklearn.preprocessing import MinMaxScaler
df_23 = pd.read_csv("/content/grafos_temporales_2023-10-23.csv")
df_24 = pd.read_csv("/content/grafos_temporales_2023-10-24.csv")
df_25= pd.read_csv("/content/grafos_temporales_2023-10-25.csv")
df_26 = pd.read_csv("/content/grafos_temporales_2023-10-26.csv")
df_27 = pd.read_csv("/content/grafos_temporales_2023-10-27.csv")


scaler_23 = MinMaxScaler()
df_23_col = df_23.iloc[:, 1:]
df_23_normal = pd.DataFrame(scaler_23.fit_transform(df_23_col), columns=df_23_col.columns)


scaler_24 = MinMaxScaler()
df_24_col = df_24.iloc[:, 1:]
df_24_normal = pd.DataFrame(scaler_24.fit_transform(df_24_col), columns=df_24_col.columns)

scaler_25 = MinMaxScaler()
df_25_col = df_25.iloc[:, 1:]
df_25_normal = pd.DataFrame(scaler_25.fit_transform(df_25_col), columns=df_25_col.columns)

scaler_26 = MinMaxScaler()
df_26_col = df_26.iloc[:, 1:]
df_26_normal = pd.DataFrame(scaler_26.fit_transform(df_26_col), columns=df_26_col.columns)

scaler_27 = MinMaxScaler()
df_27_col = df_27.iloc[:, 1:]  # Exclude the first column
df_27_normal = pd.DataFrame(scaler_27.fit_transform(df_27_col), columns=df_27_col.columns)

df_23 = pd.concat([df_23.iloc[:, [0]], df_23_normal], axis=1)
df_24 = pd.concat([df_24.iloc[:, [0]], df_24_normal], axis=1)
df_25 = pd.concat([df_25.iloc[:, [0]], df_25_normal], axis=1)
df_26 = pd.concat([df_26.iloc[:, [0]], df_26_normal], axis=1)
df_27 = pd.concat([df_27.iloc[:, [0]], df_27_normal], axis=1)




In [ ]:
import h5py
#Numero paradas: 146
#Numero rutas: 81
#Se creara la ventana de tiempo observando que se repite canda 81 columndas las paradas



periodo = 81 #numero de rutas
df_intento = pd.concat([df_23.iloc[:,1:], df_24.iloc[:,1:], df_25.iloc[:,1:], df_26.iloc[:,1:],
                        df_27.iloc[:,1:], df_30.iloc[:,1:], df_31.iloc[:,1:], df_01.iloc[:,1:],
                        df_02.iloc[:,1:], df_03.iloc[:,1:]],axis = 1)



dicc_mask_estacion = {}



for i in dicc_masking.keys():

  indices_columnas = []
  for inicio in range(0, df_intento.shape[1], periodo):
    tmp_array_columns = [inicio + col for col in dicc_masking[i]]
    tmp_array_values = df_intento.iloc[i, tmp_array_columns].values
    padded_array = np.pad(tmp_array_values, (0, 17 - len(tmp_array_values)), mode="constant")
    indices_columnas.append(padded_array)
  dicc_mask_estacion[i] = np.array(indices_columnas)


print(dicc_mask_estacion[0].shape)




with h5py.File('data_temporal.h5', 'w') as f:
  for key, matrix in dicc_mask_estacion.items():
    f.create_dataset(str(key), data=matrix)






(75, 17)


In [ ]:
#para cada llave se tiene un array (410, 81) corresponde a cada ventana temporal * número de paradas
periodo = 81 #numero de rutas
df_intento = pd.concat([df_23.iloc[:,1:], df_24.iloc[:,1:], df_25.iloc[:,1:], df_26.iloc[:,1:],
                        df_27.iloc[:,1:], df_30.iloc[:,1:], df_31.iloc[:,1:], df_01.iloc[:,1:],
                        df_02.iloc[:,1:], df_03.iloc[:,1:]],axis = 1)


dicc_estacion = {}
periodo = 81

for i in range(len(paradas)):
  tmp_array_values = df_intento.iloc[i, :].values

  reshaped_array = tmp_array_values.reshape(-1, periodo)

  dicc_estacion[i] = reshaped_array

print(dicc_estacion[0].shape)
print(dicc_estacion[0][10])

with h5py.File('data_temporal.h5', 'w') as f:
  for key, matrix in dicc_estacion.items():
    f.create_dataset(str(key), data=matrix)

(410, 81)
[ 43. 253.   0.   0.  15.   0. 480. 145.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [ ]:
with h5py.File('/content/data_temporal.h5', 'r') as f:
    loaded_dict = {key: f[key][...] for key in f.keys()}
print(loaded_dict)

{'0': array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0., 358.,   0., ...,   0.,   0.,   0.],
       ...,
       [ 90.,   0.,   0., ...,   0.,   0.,   0.],
       [ 76.,   0.,   0., ...,   0.,   0.,   0.],
       [ 47.,   0.,   0., ...,   0.,   0.,   0.]]), '1': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '10': array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [ 98.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [139., 244.,   0., ...,   0.,   0.,   0.],
       [142.,   0.,   0., ...,   0.,   0.,   0.],
       [ 10.,   0.,   0., ...,   0.,   0.,   0.]]), '100': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0

In [ ]:
#generacion del dataset para meterlo en red neuronal de Jordan,

with h5py.File("data_temporal.h5", "r") as h5_file:
  dataset = []
  for key in h5_file.keys():
    array = h5_file[key][()]
    for t in range(1, array.shape[0]):
      vector_t_minus_1 = array[t-1]
      vector_t = array[t]
      if vector_t_minus_1.any() and vector_t.any():
        dataset.append([t,key, vector_t_minus_1.tolist(), vector_t.tolist()])

df_dataset = pd.DataFrame(dataset, columns=["tiempo final", "Key", "Vector_t-1", "Vector_t"])

df_dataset.to_csv("dataset_serie_tiempo.csv", index=False)


print(df_dataset.head())


   tiempo final Key                                         Vector_t-1  \
0             2   0  [0.0, 0.0, 0.9900990099009901, 0.3058823529411...   
1             3   0  [0.0, 0.0, 1.0, 0.1638418079096045, 0.05202312...   
2             4   0  [0.22857142857142856, 0.9587628865979382, 0.45...   
3             5   0  [0.0, 0.0, 0.0, 0.41666666666666663, 0.3729166...   
4             6   0  [0.0, 0.0, 0.0, 0.0, 0.2785714285714286, 0.0, ...   

                                            Vector_t  
0  [0.0, 0.0, 1.0, 0.1638418079096045, 0.05202312...  
1  [0.22857142857142856, 0.9587628865979382, 0.45...  
2  [0.0, 0.0, 0.0, 0.41666666666666663, 0.3729166...  
3  [0.0, 0.0, 0.0, 0.0, 0.2785714285714286, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.24460431654676257, 0.0,...  


# Construccion de caracteristicas grafos/nodos

In [ ]:
# ===========================================================================================
# Codigo. Mi primera vez extrayendo características algebraicas de los grafos de Ricardo
# ===========================================================================================
!pip install networkx



import numpy as np
import pandas as pd
import networkx as nx
import h5py


#llamar la matriz de Ricardo
hdf5_filename = 'matrices_adyacencia.h5'
with h5py.File(hdf5_filename, 'r') as f:
    loaded_dict = {key: f[key][...] for key in f.keys()}

#¿Qué busca este código?
def extraer_caracteristicas_algebraicas(adj_matrix, top_k=3):
    """
    Calcula un conjunto de características basadas en álgebra lineal
    sobre la matriz de adyacencia (posiblemente no simétrica).

    Parámetros:
    -----------
    adj_matrix : np.array
        Matriz de adyacencia.
    top_k : int
        Número de valores propios (o singulares) principales que queremos extraer.

    Retorna:
    --------
    dict con varias características, por ejemplo:
    - 'trace'
    - 'sum_eigenvalues'
    - 'max_eigenvalue' (valor propio con mayor magnitud)
    - 'sum_singular_values'
    - 'largest_singular_value'
    - 'cholesky_norm' (solo si la descomposición es posible)
      etc.
    """
    feats = {}

    #  TRAZA
    trace_value = np.trace(adj_matrix)
    #feats['trace'] = trace_value

    #  Valores propios
    #    - Ojo, tener cuidado que en grafos dirigidos la matriz no siempre es simétrica => valores propipos pueden ser complejos
    #      Se usa np.linalg.eig. Ordenamos por magnitud descendente.
    try:
        eigenvalues, _ = np.linalg.eig(adj_matrix)
        idx = np.argsort(np.abs(eigenvalues))[::-1]
        eigenvalues_sorted = eigenvalues[idx]

        # Extraemos características simples:
        # - suma de valores propios (podría ser compleja si hay partes imaginarias)
        # - valor propio con mayor magnitud
        # - top_k valores propios por magnitud
        #feats['sum_eigenvalues_real'] = np.sum(eigenvalues.real)
        #feats['sum_eigenvalues_imag'] = np.sum(eigenvalues.imag)
        feats['max_eigenvalue_real'] = f"{eigenvalues_sorted[0].real:.4f}"
        feats['max_eigenvalue_imag'] = f"{eigenvalues_sorted[0].imag:.4f}"

        # (Opcional) almacenar el top_k en algo como:
        top_k_eigs_real = eigenvalues_sorted[:top_k].real
        top_k_eigs_imag = eigenvalues_sorted[:top_k].imag

        # Promedio del top_k real
        feats['avg_top_k_eigs_real'] = f"{np.mean(top_k_eigs_real):.4f}"
        # Magnitud promedio
        feats['avg_top_k_eigs_mag'] = f"{np.mean(np.abs(eigenvalues_sorted[:top_k])):.4f}"
    except np.linalg.LinAlgError as e:
        # Puede fallar si hay algo singular, etc.
        feats['sum_eigenvalues_real'] = 0
        feats['sum_eigenvalues_imag'] = 0
        feats['max_eigenvalue_real'] = 0
        feats['max_eigenvalue_imag'] = 0
        feats['avg_top_k_eigs_real'] = 0
        feats['avg_top_k_eigs_mag'] = 0

    #  VALORES SINGULARES (SVD)
    #     - SVD existe para cualquier matriz. np.linalg.svd retorna
    #       U, s, Vh donde s son valores singulares en orden descendente
    try:
        U, s, Vh = np.linalg.svd(adj_matrix, full_matrices=False)
        # s es un array con los valores singulares en orden descendente
        feats['sum_singular_values'] = f"{np.sum(s):.4f}"
        feats['largest_singular_value'] = s[0]  # el mayor
        # top_k
        feats['sum_top_k_singular_values'] = f"{np.sum(s[:top_k]):.4f}"
        feats['avg_top_k_singular_values'] = f"{np.mean(s[:top_k]):.4f}"
    except np.linalg.LinAlgError as e:
        feats['sum_singular_values'] = 0
        feats['largest_singular_value'] = 0
        feats['sum_top_k_singular_values'] = 0
        feats['avg_top_k_singular_values'] = 0

    # FACTORIZACIÓN DE CHOLESKY
    #     - Solo es posible si la matriz es Hermítica (simétrica real) y
    #       definida positiva. Es muy probable que falle en una matriz de adyacencia
    #       no simétrica o con valores negativos. Lo intentamos y si falla, lo registramos por si acaso:
    #try:
        # Asegurarnos de que sea simétrica: A + A^T ?
        #   (Esto ya depende de ti podríamos precondicionar pero es más trabajo jeje
    #    A_sym = 0.5 * (adj_matrix + adj_matrix.T)

        # Chequeo simple para ver si diagonal > 0, etc. No es garantía
        # de definida positiva, pero ayuda a evitar problemas.
    #    if np.allclose(A_sym, A_sym.T):
    #        L = np.linalg.cholesky(A_sym)
            # Podríamos extraer la norma de L, o la suma de sus diagonales, etc.
    #        feats['cholesky_norm'] = np.linalg.norm(L)
    #        feats['cholesky_trace'] = np.trace(L)
    #    else:
    #        feats['cholesky_norm'] = 0
    #        feats['cholesky_trace'] = 0
    #except np.linalg.LinAlgError as e:
        # Falló la Cholesky
    #    feats['cholesky_norm'] = 0
    #    feats['cholesky_trace'] = 0

    return feats


# =====================================================
# GENERAR UN DATAFRAME CON LAS FEATURES
# =====================================================
lista_features = []
lista_keys = []

for key, matrix in loaded_dict.items():
    feats = extraer_caracteristicas_algebraicas(matrix, top_k=3)
    lista_features.append(feats)
    print(feats)
    lista_keys.append(key)

#print(lista_keys)
#print(lista_features)

{'max_eigenvalue_real': '-75.5331', 'max_eigenvalue_imag': '0.0000', 'avg_top_k_eigs_real': '21.2470', 'avg_top_k_eigs_mag': '71.6024', 'sum_singular_values': '6158.3347', 'largest_singular_value': 796.655508987417, 'sum_top_k_singular_values': '2053.9819', 'avg_top_k_singular_values': '684.6606'}
{'max_eigenvalue_real': '551.7269', 'max_eigenvalue_imag': '0.0000', 'avg_top_k_eigs_real': '1.9480', 'avg_top_k_eigs_mag': '530.7128', 'sum_singular_values': '28396.3790', 'largest_singular_value': 3479.1381896946264, 'sum_top_k_singular_values': '6111.2070', 'avg_top_k_singular_values': '2037.0690'}
{'max_eigenvalue_real': '5.5350', 'max_eigenvalue_imag': '683.1522', 'avg_top_k_eigs_real': '3.6913', 'avg_top_k_eigs_mag': '670.7033', 'sum_singular_values': '50687.6657', 'largest_singular_value': 3204.8263692000914, 'sum_top_k_singular_values': '7205.8893', 'avg_top_k_singular_values': '2401.9631'}
{'max_eigenvalue_real': '653.4841', 'max_eigenvalue_imag': '0.0000', 'avg_top_k_eigs_real': '21

In [ ]:
from sklearn.preprocessing import StandardScaler
df_algebra = pd.DataFrame(lista_features, index=lista_keys)
df_algebra.reset_index(inplace=True)
df_algebra.rename(columns={'index': 'id_matriz'}, inplace=True)


numeric_columns = df_algebra.columns.difference(['id_matriz'])


scaler_algebra = StandardScaler()


df_algebra[numeric_columns] = scaler_algebra.fit_transform(df_algebra[numeric_columns])

print(df_algebra)


df_algebra.to_csv("caracteristicas_algebraicas.csv", index=False)



   id_matriz  max_eigenvalue_real  max_eigenvalue_imag  avg_top_k_eigs_real  \
0          0            -0.376025            -0.677772             0.066177   
1          1             1.003591            -0.677772            -0.083253   
2         10            -0.197722             1.109520            -0.069755   
3         11             1.227399            -0.677772             1.537867   
4         12            -0.351333             0.248874            -1.240831   
..       ...                  ...                  ...                  ...   
70        72             0.335388            -0.677772            -0.596985   
71        73            -0.212100            -0.035171            -0.142273   
72        74            -0.482363            -0.677772            -0.275742   
73         8             0.264341             0.108895             0.457280   
74         9            -0.203660             0.593000            -0.083704   

    avg_top_k_eigs_mag  sum_singular_values  larges

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import h5py


hdf5_filename = 'matrices_adyacencia.h5'

with h5py.File(hdf5_filename, 'r') as f:
    loaded_dict = {key: f[key][...] for key in f.keys()}

# loaded_dict es un diccionario donde:
# - la key: es el índice o nombre de la matriz
# - el value: es la matriz de adyacencia correspondiente



def extraer_caracteristicas_grafo(adj_matrix):
    """
    Dada una matriz de adyacencia (adj_matrix),
    construye un grafo dirigido (DiGraph) y extrae
    algunas características topológicas de ejemplo.
    Retorna una lista o un diccionario con dichas features.
    """
    # Crea un grafo dirigido a partir de la matriz
    # En caso de que no quieras hacerlo dirgido  usa nx.from_numpy_array(adj_matrix)
    G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)

    # Algunas características que podríamos extraer:

    #  Número de nodos y de aristas
    num_nodos = G.number_of_nodes()
    num_aristas = G.number_of_edges()

    #  Densidad (para grafos dirigidos, la fórmula difiere ligeramente)
    densidad = f"{nx.density(G):.4f}"

    # Grado medio (out_degree e in_degree pueden ser distintos en grafos dirigidos)
    out_degrees = [val for (_, val) in G.out_degree()]
    in_degrees  = [val for (_, val) in G.in_degree()]

    avg_out_degree = f"{np.mean(out_degrees):.4f}"
    avg_in_degree  = f"{np.mean(in_degrees):.4f}"

    # . Transitividad (clustering global) - en grafos dirigidos networkx
    #   la llama triadic_census, pero no es exactamente lo mismo que en no dirigidos.
    #   Otra métrica de ejemplo: nx.average_clustering(G.to_undirected()) si quieres
    #   considerarlo como no dirigido.
    #   Aquí haremos un proxy rápido usando .to_undirected():
    clustering_undirected =  f"{nx.average_clustering(G.to_undirected()):.4f}"

    #  Número de componentes fuertemente conectados
    # (devuelve un generador con las "componentes", de las cuales contamos cuántas hay)
    num_scc =  f"{nx.number_strongly_connected_components(G):.4f}"

    # (Opcional) Podríamos extraer eigenvector centrality, betweenness, etc.
    #    ¡Cuidado con el costo computacional en grafos grandes!
    #    Ejemplo (sólo si el grafo no es enorme):
    # centralidad_betweenness = nx.betweenness_centrality(G)
    # avg_betweenness = np.mean(list(centralidad_betweenness.values()))

    # Retornamos un diccionario de características
    features = {
        #'num_nodos': num_nodos,
        'num_aristas': num_aristas,
        'densidad': densidad,
        'avg_out_degree': avg_out_degree,
        'avg_in_degree': avg_in_degree,
        'clustering_undirected': clustering_undirected,
        'num_scc': num_scc
        # 'avg_betweenness': avg_betweenness,  # si lo calculamos
    }
    return features

# Iteramos sobre todas las matrices de adyacencia para construir un DataFrame de features
lista_features = []
lista_keys = []

for key, matrix in loaded_dict.items():
    f = extraer_caracteristicas_grafo(matrix)
    print(f)
    lista_features.append(f)
    lista_keys.append(key)




{'num_aristas': 89, 'densidad': '0.0043', 'avg_out_degree': '0.6138', 'avg_in_degree': '0.6138', 'clustering_undirected': '0.0189', 'num_scc': '136.0000'}
{'num_aristas': 410, 'densidad': '0.0196', 'avg_out_degree': '2.8276', 'avg_in_degree': '2.8276', 'clustering_undirected': '0.2265', 'num_scc': '15.0000'}
{'num_aristas': 567, 'densidad': '0.0272', 'avg_out_degree': '3.9103', 'avg_in_degree': '3.9103', 'clustering_undirected': '0.3171', 'num_scc': '4.0000'}
{'num_aristas': 542, 'densidad': '0.0260', 'avg_out_degree': '3.7379', 'avg_in_degree': '3.7379', 'clustering_undirected': '0.2784', 'num_scc': '6.0000'}
{'num_aristas': 484, 'densidad': '0.0232', 'avg_out_degree': '3.3379', 'avg_in_degree': '3.3379', 'clustering_undirected': '0.2477', 'num_scc': '6.0000'}
{'num_aristas': 383, 'densidad': '0.0183', 'avg_out_degree': '2.6414', 'avg_in_degree': '2.6414', 'clustering_undirected': '0.1930', 'num_scc': '21.0000'}
{'num_aristas': 205, 'densidad': '0.0098', 'avg_out_degree': '1.4138', 'a

In [ ]:
df_features = pd.DataFrame(lista_features, index=lista_keys)
df_features.reset_index(inplace=True)
df_features.rename(columns={'index': 'id_matriz'}, inplace=True)

numeric_columns = df_features.columns.difference(['id_matriz'])


scaler_networkx = StandardScaler()


df_features[numeric_columns] = scaler_networkx.fit_transform(df_features[numeric_columns])

print(df_features)
df_features.to_csv("caracteristicas_networkx.csv", index=False)

   id_matriz  num_aristas  densidad  avg_out_degree  avg_in_degree  \
0          0    -2.777015 -2.771017       -2.777016      -2.777016   
1          1    -0.285061 -0.289558       -0.285048      -0.285048   
2         10     0.933745  0.943063        0.933695       0.933695   
3         11     0.739668  0.748438        0.739633       0.739633   
4         12     0.289408  0.294315        0.289372       0.289372   
..       ...          ...       ...             ...            ...   
70        72     0.320461  0.326753        0.320440       0.320440   
71        73    -0.463612 -0.467963       -0.463576      -0.463576   
72        74    -1.767813 -1.765458       -1.767869      -1.767869   
73         8     0.343750  0.342971        0.343741       0.343741   
74         9     0.669800  0.667345        0.669842       0.669842   

    clustering_undirected   num_scc  
0               -3.057748  3.342325  
1               -0.188898 -0.173159  
2                1.063114 -0.492749  
3      

In [ ]:
with h5py.File(hdf5_filename, 'r') as f:
    loaded_dict = {key: f[key][...] for key in f.keys()}

# Función para extraer métricas por nodo

def transform_weights(A, epsilon=1e-5):
    min_weight = min(data['weight'] for _, _, data in A.edges(data=True)) #Ppermite obtener los pesos positivos manteniendo la importancia
    for u, v, data in A.edges(data=True):
        data['weight'] = data['weight'] - min_weight + epsilon
    return A

def extraer_features_por_nodo(adj_matrix, graph_id=None):
    """
    Dada una matriz de adyacencia (posiblemente dirigida),
    construye un DiGraph de NetworkX y extrae métricas por nodo.

    Parámetros:
    -----------
    adj_matrix : np.array
        Matriz de adyacencia del grafo (posiblemente no simétrica).
    graph_id : str o int
        Identificador opcional para el grafo (por ejemplo, fecha/hora).

    Retorna:
    --------
    df_nodos : pd.DataFrame
        DataFrame con columnas:
          ['graph_id', 'node_id', 'in_degree', 'out_degree',
           'betweenness', 'closeness', 'pagerank', ...]
        donde cada fila es un nodo del grafo.
    """
    # Crear el grafo dirigido a partir de la matriz
    G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)

    # Número de nodos en el grafo
    num_nodos = G.number_of_nodes()
    nodos = list(G.nodes())

    #  In-degree y Out-degree
    in_degs = dict(G.in_degree())
    out_degs = dict(G.out_degree())

    #  Betweenness centrality (centralidad de intermediación)
    #    Para grafos dirigidos, networkx usa la variante "directed=True"
    betweenness_dict = nx.betweenness_centrality(G, normalized=True)

    # Closeness centrality (centralidad de cercanía)
    closeness_dict = nx.closeness_centrality(G)

    #  PageRank (típico para grafos dirigidos)
    #    Ajustar "alpha" según convenga, por defecto 0.85
    nueva_G = transform_weights(G)
    pagerank_scores = nx.pagerank(nueva_G, weight='weight')

    #  (Opcional) Eigenvector centrality para grafo dirigido:
    #    networkx tiene eigenvector_centrality_numpy para grafos dirigidos,
    #    pero a veces requiere iteraciones grandes o puede no converger.
    #try:
    #    eigen_centr_dict = nx.eigenvector_centrality_numpy(G)
    #except:
        # Si falla (por no convergencia o singularidad),
        # ponemos 0 o np.nan
    #    eigen_centr_dict = {n: 0 for n in nodos}

    # Creamos un DataFrame para almacenar todo
    data_nodos = []
    for node in nodos:
        data_nodos.append({
            'graph_id': graph_id,
            'node_id': node,
            'in_degree': f"{in_degs[node]:.4f}",
            'out_degree': f"{out_degs[node]:.4f}",
            'betweenness': f"{betweenness_dict[node]:.4f}",
            'closeness': f"{closeness_dict[node]:.4f}",
            'pagerank': f"{pagerank_scores[node]:.4f}",
#            'eigenvector_centr': eigen_centr_dict[node]
        })

    df_nodos = pd.DataFrame(data_nodos)
    return df_nodos

In [ ]:
from sklearn.preprocessing import StandardScaler
df_features_nodo_total = []

for key, matrix in loaded_dict.items():
    # key: nombre o índice de la matriz de adyacencia
    # matrix: matriz de adyacencia (np.array)
    df_nodo = extraer_features_por_nodo(matrix, graph_id=key)
    df_features_nodo_total.append(df_nodo)

df_features_nodo_total = pd.concat(df_features_nodo_total, ignore_index=True)

numeric_columns = df_features_nodo_total.columns.difference(['graph_id', 'node_id'])


scaler_nodos = StandardScaler()

# Standardize the numerical columns
df_features_nodo_total[numeric_columns] = scaler_nodos.fit_transform(df_features_nodo_total[numeric_columns])

print(df_features_nodo_total)
df_features_nodo_total.to_csv("caracteristicas_nodo.csv", index=False)

      graph_id  node_id  in_degree  out_degree  betweenness  closeness  \
0            0        0  -1.526236   -1.534045    -0.751947  -2.115809   
1            0        1  -1.526236   -1.534045    -0.751947  -2.115809   
2            0        2  -1.526236   -1.534045    -0.751947  -2.115809   
3            0        3  -1.030838   -1.534045    -0.751947  -1.958012   
4            0        4  -1.526236   -1.534045    -0.751947  -2.115809   
...        ...      ...        ...         ...          ...        ...   
10870        9      140  -0.040042   -0.040247    -0.135651   1.282904   
10871        9      141  -1.030838   -1.036112    -0.751947   0.713923   
10872        9      142  -0.535440   -0.538179    -0.579178   0.331568   
10873        9      143   0.950754    0.457686    -0.225904   0.779166   
10874        9      144   1.446152    1.453551     1.365120   1.502910   

       pagerank  
0     -1.159951  
1     -1.159951  
2     -1.159951  
3      0.442809  
4     -1.159951  
...

In [ ]:
df_algebraicas = pd.read_csv('caracteristicas_algebraicas.csv')
df_networkx = pd.read_csv('caracteristicas_networkx.csv')
df_nodo = pd.read_csv('caracteristicas_nodo.csv')


df_algebraicas["id_matriz"] = df_algebraicas["id_matriz"].apply(lambda x: x + 1)
df_networkx["id_matriz"] = df_networkx["id_matriz"].apply(lambda x: x + 1)
df_nodo["graph_id"] = df_nodo["graph_id"].apply(lambda x: x + 1)

dicc_grafo_pesos = {}
for vector in df_algebraicas["id_matriz"]:

  vec_alg = df_algebraicas[df_algebraicas["id_matriz"] == vector].values[0]
  vec_net = df_networkx[df_networkx["id_matriz"] == vector].values[0]
  dicc_grafo_pesos[int(vector)] = np.concatenate((vec_alg[1:], vec_net[1:]))

print(dicc_grafo_pesos)




{1: array([-0.37602539, -0.67777155,  0.06617681, -1.28955451, -1.58348297,
       -1.03090545, -0.9834543 , -0.98345435, -2.77701549, -2.771017  ,
       -2.77701625, -2.77701625, -3.05774816,  3.3423247 ]), 2: array([ 1.00359101, -0.67777155, -0.08325302,  0.09851342,  0.0937727 ,
        1.99772015,  1.25568906,  1.25568906, -0.28506097, -0.28955765,
       -0.28504808, -0.28504808, -0.1888982 , -0.17315938]), 11: array([-0.19772152,  1.10952032, -0.06975486,  0.52175873,  1.77504407,
        1.68801161,  1.85983365,  1.85983365,  0.93374545,  0.94306268,
        0.93369523,  0.93369523,  1.06311435, -0.49274884]), 12: array([ 1.22739917, -0.67777155,  1.53786667,  0.40044205,  1.10331264,
        1.70737417,  1.31235507,  1.31235506,  0.739668  ,  0.74843841,
        0.73963284,  0.73963284,  0.52831429, -0.43464167]), 13: array([-0.35133279,  0.24887384, -1.2408313 , -0.46396607, -0.55708533,
        0.18649412, -0.20417862, -0.20417867,  0.2894083 ,  0.29431514,
        0.2893720

In [ ]:
df_nodo.head()

,graph_id,node_id,in_degree,out_degree,betweenness,closeness,pagerank
0,1,0,-1.526236,-1.534045,-0.751947,-2.115809,-1.159951
1,1,1,-1.526236,-1.534045,-0.751947,-2.115809,-1.159951
2,1,2,-1.526236,-1.534045,-0.751947,-2.115809,-1.159951
3,1,3,-1.030838,-1.534045,-0.751947,-1.958012,0.442809
4,1,4,-1.526236,-1.534045,-0.751947,-2.115809,-1.159951


In [ ]:
df_tiempo = pd.read_csv("/content/dataset_serie_tiempo.csv")

df_tiempo.head()
graph_properties = []

for tiempo, nodo in zip(df_tiempo['tiempo final'], df_tiempo['Key']):
  mask_nodo = (df_nodo["node_id"] == nodo) & (df_nodo["graph_id"] == tiempo)
  carac_nodo = df_nodo[mask_nodo].values[0][2:]
  carac_grafo = dicc_grafo_pesos[tiempo]
  carac_grafo_completo = np.concatenate((carac_nodo, carac_grafo)).tolist()
  graph_properties.append(carac_grafo_completo)

df_tiempo['graph properties_t-1'] = graph_properties
print(df_tiempo.head())
df_tiempo.to_csv("dataset_serie_tiempo_graph_properties.csv", index=False)


   tiempo final  Key                                         Vector_t-1  \
0             2    0  [0.0, 0.0, 0.9900990099009901, 0.3058823529411...   
1             3    0  [0.0, 0.0, 1.0, 0.1638418079096045, 0.05202312...   
2             4    0  [0.22857142857142856, 0.9587628865979382, 0.45...   
3             5    0  [0.0, 0.0, 0.0, 0.41666666666666663, 0.3729166...   
4             6    0  [0.0, 0.0, 0.0, 0.0, 0.2785714285714286, 0.0, ...   

                                            Vector_t  \
0  [0.0, 0.0, 1.0, 0.1638418079096045, 0.05202312...   
1  [0.22857142857142856, 0.9587628865979382, 0.45...   
2  [0.0, 0.0, 0.0, 0.41666666666666663, 0.3729166...   
3  [0.0, 0.0, 0.0, 0.0, 0.2785714285714286, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.24460431654676257, 0.0,...   

                                graph properties_t-1  
0  [-0.0400418190494793, -0.5381794037391997, -0....  
1  [0.9507540448028254, 0.9556187115399726, -0.63...  
2  [1.4461519767289777, 0.9556187115399726, -0.

In [ ]:
import ast
import pandas as pd
import numpy as np

df_tiempo_grafos = pd.read_csv("/content/dataset_serie_tiempo_graph_properties.csv")

df_tiempo_grafos['Vector_t-1'] = df_tiempo_grafos['Vector_t-1'].apply(ast.literal_eval)
df_tiempo_grafos['Vector_t'] = df_tiempo_grafos['Vector_t'].apply(ast.literal_eval)
df_tiempo_grafos['graph properties_t-1'] = df_tiempo_grafos['graph properties_t-1'].apply(ast.literal_eval)


vectores_t_1 = np.array(df_tiempo_grafos['Vector_t-1'].tolist())
vectores_t = np.array(df_tiempo_grafos['Vector_t'].tolist())
grafos_t_1 = np.array(df_tiempo_grafos['graph properties_t-1'].tolist())


data = np.array([(vectores_t_1[i], grafos_t_1[i], vectores_t[i]) for i in range(len(vectores_t))], dtype=object)


In [ ]:
data[51]

array([array([0.        , 0.        , 0.97826087, 0.08796296, 0.        ,
              0.        , 1.        , 0.        , 0.        , 0.        ,
              0.        , 0.        , 0.        , 0.        , 0.        ,
              0.        , 0.        ])                                   ,
       array([-0.04004182, -1.03611211, -0.72873939, -0.18582577, -0.13750065,
               0.80902432, -0.67777155, -0.09807289, -0.23326418,  0.12296359,
               2.04719383,  1.37855023,  1.37855028, -0.1453252 , -0.14358946,
              -0.14535468, -0.14535468, -0.10045581, -0.14410579])            ,
       array([0.        , 0.92592593, 0.87221571, 0.        , 0.03717472,
              0.        , 0.3929471 , 0.92946058, 0.        , 0.        ,
              0.        , 0.        , 0.        , 0.        , 0.        ,
              0.        , 0.        ])                                   ],
      dtype=object)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


class TimeSeriesDataset(Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    x, y, z = self.data[idx]
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
    y = torch.tensor(y, dtype=torch.float32)
    z = torch.tensor(z, dtype=torch.float32).unsqueeze(0)  #no se si aca se deba arreglar
    return x, y, z


dataset = TimeSeriesDataset(data)  # Instantiate the dataset

# Define sizes for splits
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size  # To account for rounding

# Randomly split the dataset
train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

# Check the lengths of the splits
print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))


batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


for x_batch, y_batch, z_batch in train_loader:
  print(f"x batch shape: {x_batch.shape}")
  print(f"y batch shape: {y_batch.shape}")
  print(f"z batch shape: {z_batch.shape}")
  break

Train dataset size: 7297
Validation dataset size: 912
Test dataset size: 913
x batch shape: torch.Size([16, 1, 17])
y batch shape: torch.Size([16, 19])
z batch shape: torch.Size([16, 1, 17])


In [ ]:
class LayerNorm(nn.Module):

  def __init__(self, eps = 1E-6):
    super().__init__()
    self.alpha = nn.Parameter(torch.ones(1))
    self.beta = nn.Parameter(torch.zeros(1))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim=True) #solo la ultima dimenasion, no quermos el batch
    std = x.std(-1, keepdim=True)
    return self.alpha * (x - mean) / (std + 1E-6) + self.beta


class EhlmanRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, z_size, num_layers=1, dropout = 0.2): #ver que pasa si despues se cambia a 2
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    # x --> (batch, seq_len, hidden_size)


    self.rnn = nn.RNN(input_size, hidden_size, num_layers, nonlinearity="relu", batch_first=True)
    self.linear1 = nn.Linear(hidden_size, output_size)
    self.linear2 = nn.Linear(output_size + z_size, output_size + z_size)
    self.linear3 = nn.Linear(output_size + z_size, output_size)
    self.norm = LayerNorm()

  def forward(self, x, y):
    residual = x.squeeze(1)
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    out, _ = self.rnn(x, h0)
    # out --> (batch, seq_len, hidden_size)
    out = self.linear1(out[:, -1, :])
    # out --> (batch, seq_len, output)
    out += residual
    #out = self.residual(x, self.residual)
    #print(f"After residual connection (out.shape): {out.shape}")
    concatenated = torch.cat((y, out), dim=-1)
    normalized_out = self.norm(concatenated)
    out = self.linear2(normalized_out)
    out = nn.Tanh()(out)
    out = self.linear3(out)
    out = nn.Sigmoid()(out)

    return out





input_size = 17
hidden_size = 36
output_size = 17
z_size = 19

# Create the model instance
model = EhlmanRNN(input_size=input_size, hidden_size=hidden_size, output_size=output_size, z_size=z_size, num_layers=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer with L2 regularization (weight decay)
optimizer = optim.Adam(model.parameters(), lr=1E-4, weight_decay=1e-4)

# Training loop
epochs = 20
for epoch in range(epochs):
    running_loss = 0.0
    for x,y,z in train_loader:
      x, y, z = x.to(device), y.to(device), z.to(device)
      optimizer.zero_grad()
      output = model(x,y)
      loss = criterion(output, z)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")



    model.eval()
    val_loss = 0.0
    with torch.no_grad():
      for x, y, z in val_loader:
        x, y, z = x.to(device), y.to(device), z.to(device)
        output = model(x, y)
        loss = criterion(output, z)
        val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Validation Loss: {val_loss/len(val_loader):.4f}")

model.eval()
test_loss = 0.0
i = 0
with torch.no_grad():
  for x, y, z in val_loader:
    x, y, z = x.to(device), y.to(device), z.to(device)
    output = model(x, y)
    if i % 10 == 0:
      print(f"el output es {output}")
      print(f"el valor real es {z}")
    loss = criterion(output, z)
    test_loss += loss.item()
    i += 1

print(test_loss / len(test_loader))

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([16, 1, 17])) that is different to the input size (torch.Size([16, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1, 1, 17])) that is different to the input size (torch.Size([1, 17])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Loss: 0.1609
Epoch [1/20], Validation Loss: 0.0997
Epoch [2/20], Loss: 0.0858
Epoch [2/20], Validation Loss: 0.0810
Epoch [3/20], Loss: 0.0794
Epoch [3/20], Validation Loss: 0.0793
Epoch [4/20], Loss: 0.0786
Epoch [4/20], Validation Loss: 0.0789
Epoch [5/20], Loss: 0.0783
Epoch [5/20], Validation Loss: 0.0788
Epoch [6/20], Loss: 0.0783
Epoch [6/20], Validation Loss: 0.0788
Epoch [7/20], Loss: 0.0782
Epoch [7/20], Validation Loss: 0.0787
Epoch [8/20], Loss: 0.0784
Epoch [8/20], Validation Loss: 0.0786
Epoch [9/20], Loss: 0.0782
Epoch [9/20], Validation Loss: 0.0787
Epoch [10/20], Loss: 0.0782
Epoch [10/20], Validation Loss: 0.0786
Epoch [11/20], Loss: 0.0781
Epoch [11/20], Validation Loss: 0.0787
Epoch [12/20], Loss: 0.0781
Epoch [12/20], Validation Loss: 0.0786
Epoch [13/20], Loss: 0.0783
Epoch [13/20], Validation Loss: 0.0786
Epoch [14/20], Loss: 0.0781
Epoch [14/20], Validation Loss: 0.0786
Epoch [15/20], Loss: 0.0781
Epoch [15/20], Validation Loss: 0.0786
Epoch [16/20]

In [ ]:
!pip install netron
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.7/702.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


batch_size = 16
seq_len = 1
input_size = 17
z_size = 19


x_dummy = torch.randn(batch_size, seq_len, input_size).to(device)
y_dummy = torch.randn(batch_size, z_size).to(device)


onnx_model_path = "ehlman_rnn.onnx"

torch.onnx.export(
    model,
    (x_dummy, y_dummy),  # Inputs to the model
    onnx_model_path,
    input_names=["input_x", "input_y"],  # Define input names
    output_names=["output"],            # Define output name
    dynamic_axes={
        "input_x": {0: "batch_size", 1: "seq_len"},  # Dynamic batch and sequence dimensions
        "input_y": {0: "batch_size"},
        "output": {0: "batch_size"}
    },
    opset_version=12,  # Specify ONNX opset version
    verbose=True       # Print debugging information during export
)



/usr/local/lib/python3.11/dist-packages/torch/onnx/symbolic_opset9.py:4279: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with RNN_RELU can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
